In [1]:
import pandas as pd

from IPython.display import display

In [15]:
df_all = pd.read_csv('./results/sachs/joint_benchmarks.csv', index_col=0)
df_all['id'] = df_all.apply(lambda x: x['id'].upper(), axis=1)

In [7]:
def best_h(df):
    return df.groupby(['id'], as_index=False).apply(lambda x: x.loc[x['SHD_pattern'].idxmin()])

def worst_h(df):
    return df.groupby(['id'], as_index=False).apply(lambda x: x.loc[x['SHD_pattern'].idxmax()])

In [16]:
df_best = best_h(df_all)
df_best['SHD'] = df_best['SHD_pattern']

df_worst = worst_h(df_all)
df_worst['SHD'] = df_worst['SHD_pattern']

In [17]:
df_best[['id', 'SHD']]

,id,SHD
0,ANM,15
1,CAM,9
2,FCI,9
3,FGES,9
4,LINGAM,9
5,NOTEARS,8
6,NOTEARS_MLP,8
7,PC,9


In [18]:
df_merged = df_best.merge(df_worst, on='id', suffixes=['_best', '_worst'])

In [19]:
display(df_merged[['id', 'SHD_best', 'SHD_worst']])

,id,SHD_best,SHD_worst
0,ANM,15,25
1,CAM,9,32
2,FCI,9,21
3,FGES,9,15
4,LINGAM,9,12
5,NOTEARS,8,39
6,NOTEARS_MLP,8,33
7,PC,9,22
